In [1]:
import numpy as np
import random
from copy import deepcopy as cp

In [2]:
#reads the dataset
#operationTimes[i][j]: operation time of job i on machine j
def readSet(setName):
    f = open(setName, 'r')
    readingParams=True
    operationTimes=[]
    for line in f:
        line=line.split()
        if readingParams:
            numberOfJobs=int(line[0])
            numberOfMachines=int(line[1])
            readingParams=False
        else:
            machines=[]
            for i in range(0,numberOfMachines):
                machines.append(int(line[(2*i)+1]))
            operationTimes.append(machines)
    return operationTimes,numberOfJobs,numberOfMachines   

#generates population according to number of jobs and population size
def generatePopulation(numberOfJobs, populationSize):
    np.random.seed(1)
    population=[]
    for size in range (0,populationSize):
        population.append(np.random.permutation(numberOfJobs))
    return population

#calculates completion times of all jobs and machines
#completionTimes[i][j]: completion time of ith job in individual and machine j
#for example if individual=[2,5,6,1,0,3,4,7]
#completionTimes[1][j]: completion time of 1st job in individual (which is job 2) and machine j
#completionTimes[2][j]: completion time of 2nd job in individual (which is job 5) and machine j
def calculateCompletionTime(individual,emptyCompletionTimes,operationTimes,numberOfJobs,numberOfMachines):
    completionTimes=cp(emptyCompletionTimes)
    for k in range(0,numberOfJobs):
        job=individual[k]
        if k==0:
            for l in range (0,numberOfMachines):
                if l==0:       
                    completionTimes[k][l]=operationTimes[job][l]
                else:
                    completionTimes[k][l]=operationTimes[job][l]+completionTimes[k][l-1]
        else:
            for l in range (0,numberOfMachines):
                if l==0:
                    completionTimes[k][l]=operationTimes[job][l]+completionTimes[k-1][l]
                else:
                    completionTimes[k][l]=operationTimes[job][l]+max(completionTimes[k][l-1],completionTimes[k-1][l])
    return completionTimes   

In [3]:
#SOLUTION REPRESENTATION: PERMUTATION REPRESENTATION
#iterates over data sets
for s in range(1,4):
    #reads data set
    operationTimes,numberOfJobs,numberOfMachines=readSet('instance%s.txt' % s)
    
    #optimal values for different data sets
    optimal=0
    if s==1:
        optimal=4534
    elif s==2:
        optimal=920
    elif s==3:
        optimal=1302
    
    #generates random population
    populationSize=100
    population=generatePopulation(numberOfJobs, populationSize)
    completionTimes=np.zeros((numberOfJobs,numberOfMachines))

    #parent selection: binary tournament method
    #select candidate parents randomly
    #2*m parents are selected
    m=10
    candidates=np.random.randint(populationSize, size=4*m)
    parents=[]
    for r in range(0,2*m):
        #randomly choose 2 parents
        can0=population[candidates[r*2]]
        can1=population[candidates[(r*2)+1]]
        #calculate completion times and makespans
        cT0=calculateCompletionTime(can0,completionTimes,operationTimes,numberOfJobs,numberOfMachines)
        makespan0=cT0[numberOfJobs-1][numberOfMachines-1]
        cT1=calculateCompletionTime(can1,completionTimes,operationTimes,numberOfJobs,numberOfMachines)
        makespan1=cT1[numberOfJobs-1][numberOfMachines-1] 
        if makespan0<makespan1:
            parents.append(can0)
        else:
            parents.append(can1)
    #parents: size 2*m, each element is a permutation representation of a parent